In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv('data/TSLA.csv')

# Convert date to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Set date as index
df.set_index('Date', inplace=True)

# Filter out data before 2016
df = df[df.index.year >= 2016]

# Calculate the end of week dates
df['Week'] = df.index.to_series().dt.isocalendar().week
df['Year'] = df.index.to_series().dt.isocalendar().year
df['Month'] = df.index.month  # Add Month column
df['DateStr'] = df.index.strftime('%y%m%d')
df['TradingWeek'] = (
    df['Year'].astype(str) + '-' 
    + df['Week'].astype(str).apply(lambda x: str(x).zfill(2))
)

# Filter out the end trading day of each week
weekly_df = df.groupby('TradingWeek').last()

# Calculate the last Friday's close price
weekly_df['PrevClose'] = weekly_df['Adj Close'].shift(1)

# Initialize the profit and investment column
weekly_df['Profit_No_Reinvest'] = 0.0
weekly_df['Investment_No_Reinvest'] = 1.0

weekly_df['Profit_Reinvest'] = 0.0
weekly_df['Investment_Reinvest'] = 1.0

strike_factor = 1.15
premium = 0.005
leverage = 3
for i in range(1, len(weekly_df)):
    prev_close = weekly_df.loc[weekly_df.index[i-1], 'Adj Close']
    this_close = weekly_df.loc[weekly_df.index[i], 'Adj Close']
    if this_close <= prev_close * strike_factor:
        profit = prev_close * premium
    else:
        profit = prev_close * premium - leverage * (this_close - prev_close * strike_factor)
    
    # Each week's profit is the investment times the premium
    weekly_df.loc[weekly_df.index[i], 'Profit_No_Reinvest'] = weekly_df.loc[weekly_df.index[i-1], 'Investment_No_Reinvest'] * profit

    # For reinvest, each week's profit is the investment times the premium, and then add the profit to the investment
    weekly_df.loc[weekly_df.index[i], 'Profit_Reinvest'] = weekly_df.loc[weekly_df.index[i-1], 'Investment_Reinvest'] * profit
    weekly_df.loc[weekly_df.index[i], 'Investment_Reinvest'] = weekly_df.loc[weekly_df.index[i-1], 'Investment_Reinvest'] + weekly_df.loc[weekly_df.index[i], 'Profit_Reinvest']

# Calculate the current value of 1 dollar investment in TSLA stock
weekly_df['Stock_Value'] = weekly_df['Close'] / weekly_df.loc[weekly_df.index[0], 'Close']

# Select only necessary columns
output_df = weekly_df[['DateStr', 'PrevClose', 'Close', 'Profit_No_Reinvest', 'Investment_No_Reinvest', 'Profit_Reinvest', 'Investment_Reinvest', 'Stock_Value']].reset_index()
output_df

In [51]:
# Initialize the profit and investment column
pd.set_option('display.max_rows', 500)
weekly_df['Profit_No_Reinvest'] = 0.0
weekly_df['Investment_No_Reinvest'] = 1.0 # The initial investment is 1 stock

weekly_df['Profit_Reinvest'] = 0.0
weekly_df['Investment_Reinvest'] = 1.0  # The initial investment is 1 stock

strike_factor = 1.15  # Adjust strike factor to 1.3
premium = 0.0065
leverage = 5
for i in range(1, len(weekly_df)):
    prev_close = weekly_df.loc[weekly_df.index[i-1], 'Adj Close']
    this_close = weekly_df.loc[weekly_df.index[i], 'Adj Close']
    strike_price = prev_close * strike_factor
    # Calculate One Share Profit
    if this_close < strike_price:
        profit = prev_close * premium
    else:
         # calculate loss in the future
        profit = prev_close * premium - leverage * (this_close - strike_price) 
    # Each week's profit is the calculated profit
    # Investment_No_Reinvest is the cum sum of each week profit
    weekly_df.loc[weekly_df.index[i], 'Profit_No_Reinvest'] = profit
    weekly_df.loc[weekly_df.index[i], 'Investment_No_Reinvest'] += (
        weekly_df.loc[weekly_df.index[i], 'Profit_No_Reinvest']
    )


    # For reinvest, each week's profit is the total value of all stocks at the end of the week times the premium
    weekly_df.loc[weekly_df.index[i], 'Profit_Reinvest'] = (
        weekly_df.loc[weekly_df.index[i-1], 'Investment_Reinvest'] 
        * (1 + profit/weekly_df.loc[weekly_df.index[i], 'Adj Close'])
    )
    weekly_df.loc[weekly_df.index[i], 'Investment_Reinvest'] += (
        weekly_df.loc[weekly_df.index[i], 'Profit_Reinvest']
    )
    

# Calculate the current value of 1 dollar investment in TSLA stock
weekly_df['Stock_Value'] = weekly_df['Close'] / weekly_df.loc[weekly_df.index[0], 'Close']

# Select only necessary columns
output_df = weekly_df[[
    'Profit_No_Reinvest',
    'Investment_No_Reinvest', 
    'Profit_Reinvest',
    'Investment_Reinvest', 
    'Stock_Value'
    ]].reset_index()
output_df


,TradingWeek,Profit_No_Reinvest,Investment_No_Reinvest,Profit_Reinvest,Investment_Reinvest,Stock_Value
0,2016-01,0.000000,1.000000,0.000000,1.000000,1.000000
1,2016-02,0.091433,1.091433,1.006691,2.006691,0.971517
2,2016-03,0.088829,1.088829,2.019891,3.019891,0.959953
3,2016-04,0.087772,1.087772,3.040686,4.040686,0.906161
4,2016-05,0.082853,1.082853,4.071570,5.071570,0.770616
5,2016-06,0.070460,1.070460,5.107058,6.107058,0.715829
6,2016-07,0.065451,1.065451,6.143051,7.143051,0.789479
7,2016-08,0.072185,1.072185,7.183685,8.183685,0.902085
8,2016-09,0.082481,1.082481,8.234048,9.234048,0.952796
9,2016-10,0.087117,1.087117,9.292200,10.292200,0.983412


## Optimize Trading Strategy

In [ ]:
# Earnings dates
earnings_dates = pd.to_datetime([
    '7/19/23', '4/19/23', '1/25/23', '10/19/22', '7/20/22', '4/20/22', '1/26/22', '10/20/21', 
    '7/26/21', '4/26/21', '1/27/21', '10/21/20', '7/22/20', '4/29/20', '1/29/20', '10/23/19', 
    '6/24/19', '4/24/19', '1/30/19', '10/24/18', '8/1/18', '5/2/18', '2/7/18', '11/1/17', 
    '8/2/17', '5/3/17', '2/22/17', '10/26/16', '8/3/16', '5/4/16'
])

# Function to get the first week in the data that is after each earnings date
def first_week_after_date(date):
    if date > weekly_data.index[-1]:
        return None
    return weekly_data[date:].index[0]

# Apply the function to the earnings dates
earnings_weeks = earnings_dates.map(first_week_after_date)

# Weeks that are both high return weeks and have earnings dates
high_return_earnings_weeks = weekly_data.loc[high_return_weeks.index.intersection(earnings_weeks)]

# Plot the weekly returns, highlighting the earnings weeks
plt.figure(figsize=(12, 6))
plt.plot(weekly_data.index, weekly_data['Weekly Return %'], label='All weeks')
plt.scatter(high_return_earnings_weeks.index, high_return_earnings_weeks['Weekly Return %'], 
            color='red', label='High return earnings weeks', marker='x', s=100)
plt.title('Weekly Returns of Tesla Shares')
plt.xlabel('Date')
plt.ylabel('Weekly Return %')
plt.legend()
plt.grid()
plt.show()